Importar librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost

Importar datasets limpios

In [ ]:
url = 'https://raw.githubusercontent.com/Ciarzi/DP3_GP1/main/datasets/own_data/train.csv'
df = pd.read_csv(url)

Separar en *x* (variables de estudio) e *y* (predicciones en 0 o 1)

In [ ]:
x = df.loc[:,df.columns!='flag']

In [ ]:
y = df.loc[:,'flag']

In [ ]:
x.shape, y.shape, df.shape

((4358, 10), (4358,), (4358, 11))

In [ ]:
x.head()

,loannumber,historial,referido,age,due_per_day,interes,cuenta_corriente,cuenta_otra,cuenta_ahorro,employment
0,12,0,1,50,1150,15.00,0,1,0,2
1,7,1,1,37,1483,11.25,0,1,0,2
2,3,0,1,44,766,15.00,0,0,1,2
3,9,1,1,35,1466,10.00,0,1,0,2
4,8,1,1,49,650,30.00,0,0,1,2


Separamos x en train y test (50% train y 50% test y validación 25/25).
De este modo utilizaremos el 25% del total para frenar el algoritmo cuando no esté entrenando lo suficioente y el otro 25% para validar nuevas prediciones.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state=1)
#el test size es el 50% de los casos y el random state 1 para que siempre haga de la misma forma el split

In [ ]:
x_train.shape, x_test.shape

((2179, 10), (2179, 10))

In [ ]:
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state=1)
#50% los casos de entrenamiento y 25/25% los casos de test y validación

In [ ]:
x_train.shape, x_test.shape, x_valid.shape

((2179, 10), (1090, 10), (1089, 10))

In [ ]:
xgb = xgboost.XGBClassifier()

Definimos los parámetros para afinar el resultado y recibir las mejores predicciones posibles

In [ ]:
parameters = {'nthreads' : [1], #
              'objective': ['binary:logistic'], # clasificar entre 0 y 1
              'learning_rate' : [0.05, 0.1], # tasa de aprendizaje, se entrenará con ambos valores
              'n_estimators' : [140, 150]} # número de arboles, para gestionar la duración del entrenamiento

Realizamos validaciones cruzadas con todos los parámetros para saber cual es la mejor combinación posible. Introducimos nuevos parámetros de entrenamiento para evitar sobre-entrenar el modelo, utilizando el test para que a través de una función de pérdida indique al modelo que si no consigue mejorar esta función de perdida en el numero de rondas introducido, debe dentener el entrenemiento. De este modo se consigue un entrenamiento mas rápido y se evita sobreentrenar el modelo y así poder  recibir prediciones sobre datos en los que no conocemos la respuesta.

In [ ]:
fit_params = {'early_stoppping_rounds' : 10, #si en estas rondas no mejora el score (función de pérdida) detendrá el entrenamiento
              'eval_metric' : 'logloss', #función de pérdida
              'eval_set' : [(x_test, y_test)]}  #el conjunto de test sobre el que optimizará con la función logloss, si no lo hace en 10 rondas, se detiene

Creamos el clasificador con GridSearch

In [ ]:
clf = GridSearchCV(xgb, parameters, cv=3, scoring='accuracy')

In [ ]:
clf.fit(x_train, y_train) 
#aquí deberíamos introducir clf.fit(x_train, y_train, **fit_params), pero hay un fallo que no consigo resolver


GridSearchCV(cv=3, estimator=XGBClassifier(),
             param_grid={'learning_rate': [0.05, 0.1],
                         'n_estimators': [140, 150], 'nthreads': [1],
                         'objective': ['binary:logistic']},
             scoring='accuracy')

In [ ]:
clf.best_estimator_ #mejor combinación de parámetros

XGBClassifier(learning_rate=0.05, n_estimators=140, nthreads=1)

In [ ]:
clf.best_score_ #mejor resultado dentro de las validaciones cruzadas (empezamos con 0.7746648680123734)

0.7732887206439788

Validamos con el tercer conjunto de entrenamiento (*x_ valid* e *y_valid*)

In [ ]:
best_xgb = clf.best_estimator_ 

In [ ]:
y_preds = best_xgb.predict(x_valid)

Creamos dataframe de comparación

In [ ]:
comp = pd.DataFrame ({'Datos' : y_valid, 'Predicciones' : y_preds})

In [ ]:
comp.head(20)

,Datos,Predicciones
3469,1,1
3546,0,1
3679,0,1
1859,1,1
1441,1,1
4063,1,1
3159,1,1
1717,1,1
3211,1,1
1828,1,1


In [ ]:
acc = accuracy_score(y_valid, y_preds) # comprobar el ratio de acierto (empezamos con 0.8080808080808081)
acc

0.8099173553719008